# Reglas de asociación

In [10]:
import pandas as pd
import ETL as etl
import numpy as np
import os

from mlxtend.frequent_patterns import apriori, association_rules

In [11]:
# Get query files function
def get_path(folder, file_path):
    current_directory = os.getcwd()
    return os.path.join(current_directory, '..', folder, file_path)

In [12]:
procesos_data = pd.read_csv(get_path('data', 'collected_obra_data_v3.csv'),
                 dtype = {'ID_ADJUDICACION': str},
                 parse_dates=['START_DATE','CONTRACT_DATE'])

In [13]:
def to_quantile(variable, data):
    #variable = 'NUM_PENALTIES_ENTIDAD_LAST_Y'

    percentile = [25,50,75]
    thresholds = np.percentile(data[variable], percentile)
    thresholds = [0] + list(thresholds) + [np.max(data[variable]) + 1]
    categories = np.digitize(data[variable], thresholds)

    etiquetas = ['Muy bajo','Bajo','Medio', 'Alto','Muy Alto']

    # Mapear los índices a las etiquetas personalizadas
    etiquetas_binarias = [etiquetas[i-1] for i in categories]

    data[variable + '_QUANTILE'] = etiquetas_binarias

    return data

variables = ['NUM_PENALTIES_ENTIDAD_LAST_Y', 'PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y', 'NUM_PENALTIES_ENTIDAD_ACTIVE','PROJECT_INTENSITY_NORM']

for var in variables:
    procesos_data_2 = to_quantile(var, procesos_data)

In [14]:
variables = ['NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE', 'PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE', 
             'NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE','PROJECT_INTENSITY_NORM_QUANTILE',
             'MUNICIPALITY_TYPE', 'COST_GROUP_QUANTILE', 'REGION', 'HAVE_DEVIATION']


procesos_data_2 = procesos_data_2[variables]

procesos_data_2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 8 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE          167 non-null    object
 1   PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE  167 non-null    object
 2   NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE          167 non-null    object
 3   PROJECT_INTENSITY_NORM_QUANTILE                167 non-null    object
 4   MUNICIPALITY_TYPE                              167 non-null    object
 5   COST_GROUP_QUANTILE                            167 non-null    object
 6   REGION                                         167 non-null    object
 7   HAVE_DEVIATION                                 167 non-null    bool  
dtypes: bool(1), object(7)
memory usage: 9.4+ KB


In [15]:
procesos_data_2 = pd.get_dummies(procesos_data_2, )
procesos_data_2

,HAVE_DEVIATION,NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Bajo,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Medio,PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Muy bajo,NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto,PROJECT_INTENSITY_NORM_QUANTILE_Alto,PROJECT_INTENSITY_NORM_QUANTILE_Bajo,PROJECT_INTENSITY_NORM_QUANTILE_Medio,...,MUNICIPALITY_TYPE_TYPE_6,COST_GROUP_QUANTILE_Alto,COST_GROUP_QUANTILE_Bajo,COST_GROUP_QUANTILE_Medio,COST_GROUP_QUANTILE_Muy Alto,REGION_AMAZONIA,REGION_ANDINA,REGION_CARIBE,REGION_ORINOQUIA,REGION_PACIFICA
0,False,True,False,False,False,True,True,False,True,False,...,True,False,False,True,False,False,True,False,False,False
1,True,True,False,True,False,False,True,True,False,False,...,False,False,False,False,True,False,True,False,False,False
2,True,True,True,False,False,False,True,False,True,False,...,True,True,False,False,False,False,True,False,False,False
3,False,True,False,False,True,False,True,False,False,True,...,True,True,False,False,False,False,True,False,False,False
4,True,True,False,True,False,False,True,False,False,True,...,True,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,False,True,False,False,False,True,True,False,True,False,...,True,False,False,True,False,False,True,False,False,False
163,False,True,True,False,False,False,True,False,False,True,...,True,False,False,False,True,False,True,False,False,False
164,True,True,True,False,False,False,True,False,False,True,...,True,True,False,False,False,False,False,False,False,True
165,False,True,False,False,False,True,True,False,False,True,...,False,False,True,False,False,False,True,False,False,False


In [16]:
lista_soporte = apriori(procesos_data_2, min_support = 0.03, use_colnames = True)
lista_soporte

,support,itemsets
0,0.419162,(HAVE_DEVIATION)
1,1.000000,(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto)
2,0.251497,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto)
3,0.245509,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Bajo)
4,0.251497,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Medio)
...,...,...
1302,0.041916,"(HAVE_DEVIATION, MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, REGION_ANDINA, COST_GROUP_QUANTILE_Medio)"
1303,0.035928,"(HAVE_DEVIATION, MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, REGION_ANDINA, COST_GROUP_QUANTILE_Bajo, PROJECT_INTENSITY_NORM_QUANTILE_Muy bajo)"
1304,0.035928,"(HAVE_DEVIATION, MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, REGION_ANDINA, PROJECT_INTENSITY_NORM_QUANTILE_Muy bajo, COST_GROUP_QUANTILE_Medio)"
1305,0.035928,"(PROJECT_INTENSITY_NORM_QUANTILE_Medio, MUNICIPALITY_TYPE_TYPE_6, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Medio, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, COST_GROUP_QUANTILE_Alto, REGION_ANDINA)"


In [17]:
# Calcular las métricas para las reglas de asociación y filtrar aquellas con mínimo lift
reglas = association_rules(lista_soporte, metric = "support", min_threshold = 0.1)
reglas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(HAVE_DEVIATION),(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto),0.419162,1.000000,0.419162,1.000000,1.000000,0.000000,inf,0.000000
1,(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto),(HAVE_DEVIATION),1.000000,0.419162,0.419162,0.419162,1.000000,0.000000,1.000000,0.000000
2,(HAVE_DEVIATION),(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto),0.419162,0.251497,0.161677,0.385714,1.533673,0.056259,1.218493,0.599084
3,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto),(HAVE_DEVIATION),0.251497,0.419162,0.161677,0.642857,1.533673,0.056259,1.626347,0.464889
4,(HAVE_DEVIATION),(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Bajo),0.419162,0.245509,0.125749,0.300000,1.221951,0.022841,1.077844,0.312715
...,...,...,...,...,...,...,...,...,...,...
2573,(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto),"(MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, REGION_ANDINA, COST_GROUP_QUANTILE_Bajo, PROJECT_INTENSITY_NORM_QUANTILE_Muy bajo)",1.000000,0.119760,0.119760,0.119760,1.000000,0.000000,1.000000,0.000000
2574,(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto),"(MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, REGION_ANDINA, COST_GROUP_QUANTILE_Bajo, PROJECT_INTENSITY_NORM_QUANTILE_Muy bajo)",1.000000,0.119760,0.119760,0.119760,1.000000,0.000000,1.000000,0.000000
2575,(REGION_ANDINA),"(MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, COST_GROUP_QUANTILE_Bajo, PROJECT_INTENSITY_NORM_QUANTILE_Muy bajo)",0.766467,0.137725,0.119760,0.156250,1.134511,0.014199,1.021956,0.507692
2576,(COST_GROUP_QUANTILE_Bajo),"(MUNICIPALITY_TYPE_TYPE_6, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, REGION_ANDINA, PROJECT_INTENSITY_NORM_QUANTILE_Muy bajo)",0.251497,0.203593,0.119760,0.476190,2.338936,0.068557,1.520414,0.764800


In [18]:
pd.set_option('display.max_colwidth', None)
# Filtrar reglas por ítem
reglas[(reglas['consequents'] == {'HAVE_DEVIATION'}) & (reglas['lift'] > 1)].sort_values(by='lift', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3,(PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto),(HAVE_DEVIATION),0.251497,0.419162,0.161677,0.642857,1.533673,0.056259,1.626347,0.464889
210,"(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.251497,0.419162,0.161677,0.642857,1.533673,0.056259,1.626347,0.464889
144,"(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.251497,0.419162,0.161677,0.642857,1.533673,0.056259,1.626347,0.464889
811,"(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto)",(HAVE_DEVIATION),0.251497,0.419162,0.161677,0.642857,1.533673,0.056259,1.626347,0.464889
1950,"(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, PROJECT_INTENSITY_NORM_QUANTILE_Alto, COST_GROUP_QUANTILE_Muy Alto, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.161677,0.419162,0.101796,0.629630,1.502116,0.034028,1.568263,0.398739
1048,"(COST_GROUP_QUANTILE_Muy Alto, PROJECT_INTENSITY_NORM_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto)",(HAVE_DEVIATION),0.161677,0.419162,0.101796,0.629630,1.502116,0.034028,1.568263,0.398739
978,"(PROJECT_INTENSITY_NORM_QUANTILE_Alto, COST_GROUP_QUANTILE_Muy Alto, NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto)",(HAVE_DEVIATION),0.161677,0.419162,0.101796,0.629630,1.502116,0.034028,1.568263,0.398739
282,"(PROJECT_INTENSITY_NORM_QUANTILE_Alto, COST_GROUP_QUANTILE_Muy Alto)",(HAVE_DEVIATION),0.161677,0.419162,0.101796,0.629630,1.502116,0.034028,1.568263,0.398739
1890,"(NUM_PENALTIES_ENTIDAD_LAST_Y_QUANTILE_Alto, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, REGION_ANDINA)",(HAVE_DEVIATION),0.185629,0.419162,0.113772,0.612903,1.462212,0.035964,1.500499,0.388158
1021,"(NUM_PENALTIES_ENTIDAD_ACTIVE_QUANTILE_Muy Alto, PROPORTION_DEVIATIONS_ENTIDAD_LAST_Y_QUANTILE_Alto, REGION_ANDINA)",(HAVE_DEVIATION),0.185629,0.419162,0.113772,0.612903,1.462212,0.035964,1.500499,0.388158
